In [1]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# data
dataset_path = "./data/"
faceData = []
labels = []
classId = 0
nameMap = {}

In [3]:
for f in os.listdir(dataset_path):
    if f.endswith(".npy"):
        nameMap[classId] = f[:-4]
        # X-value
        dataItem = np.load(dataset_path + f)
        m = dataItem.shape[0]
        faceData.append(dataItem)

        # Y-values
        target = classId * np.ones((m,))
        classId += 1
        labels.append(target)

# print(faceData)
# print(labels)

XT = np.concatenate(faceData, axis=0)
yT = np.concatenate(labels,axis=0).reshape((-1,1))
print(XT.shape)
print(yT.shape)
print(nameMap)

(60, 30000)
(60, 1)
{0: 'ravi', 1: 'sindhu', 2: 'dheeraj'}


In [4]:
# # KNN Algorithm


# def dist(p,q):
#     return np.sqrt(np.sum((p - q)**2))

# def knn(X,y,xt,k=5):

#     m = X.shape[0]
#     dlist = []
    
#     for i in range(m):
#         d = dist(X[i],xt)
#         dlist.append((d,y[i]))

#     dlist = sorted(dlist)
#     dlist = np.array(dlist[:k])
#     labels = dlist[:,1]

#     labels, cnts = np.unique(labels,return_counts=True)
#     idx = cnts.argmax()
#     pred = labels[idx]

#     return int(pred)

In [5]:
# predictions

cam = cv2.VideoCapture(0)
model = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(XT,yT)
while True:
    success, img = cam.read()
    if not success:
        print("Camera reading Failed !")
    
    faces = model.detectMultiScale(img, 1.3, 5)
    for f in faces:
        x,y,w,h = f

        # crop and save the largest face
        cropped_face = img[y:y+h,x:x+w]
        cropped_face = cv2.resize(cropped_face, (100, 100)).flatten().reshape(1,-1)
        
        # predict name using knn


        text = knn.predict(cropped_face)
        cv2.putText(img, nameMap[text[0]], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.rectangle(img, (x, y), (x + w, y + w), (0, 0, 255), 2)
        
    cv2.imshow("prediction Window", img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cam.destroyAllWinodws()
    

[ WARN:0@0.494] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2023-12-26 18:55:00.054 python[4600:584110] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
/Users/dheeraj/anaconda3/envs/tensorflow/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


AttributeError: 'cv2.VideoCapture' object has no attribute 'destroyAllWinodws'